In [69]:
import altair as alt
import pandas as pd
import os

In [70]:
air_df = pd.read_csv('../data/airplanes_final.csv')
data = air_df.copy()

In [72]:
data.head()

,Date,Location,Operator,Type,Aboard,Fatalities,Ground,Summary,City,State,State_name
0,1908-09-17,"Fort Myer, Virginia",Military - U.S. Army,Wright Flyer III,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",Fort Myer,VA,Virginia
1,1912-07-12,"AtlantiCity, New Jersey",Military - U.S. Navy,Dirigible,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,AtlantiCity,NJ,New Jersey
2,1918-12-16,"Elizabeth, New Jersey",US Aerial Mail Service,De Havilland DH-4,1.0,1.0,0.0,NaN,Elizabeth,NJ,New Jersey
3,1919-05-25,"Cleveland, Ohio",US Aerial Mail Service,De Havilland DH-4,1.0,1.0,0.0,Caught fire in midair. The pilot leaped from t...,Cleveland,OH,Ohio
4,1919-07-19,"Dix Run, Pennsylvania",US Aerial Mail Service,De Havilland DH-4,1.0,1.0,0.0,NaN,Dix Run,PA,Pennsylvania


In [73]:
# aggregate the data by state
state_counts = data.groupby('State')['Location'].count().reset_index()
state_counts = state_counts.rename(columns={'Location': 'Crash Count'})

In [77]:
state_counts.head()

,State,Crash Count,State_name
0,AK,176,Alaska
1,AL,13,Alabama
2,AR,13,Arkansas
3,AZ,32,Arizona
4,CA,143,California


In [76]:
states = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}
state_counts['State_name'] = state_counts['State'].map(states)

In [78]:


# create a selection that filters the data by state
state_selection = alt.selection_single(fields=['State'], empty='all')
color_condition = alt.condition(state_selection,
                                alt.Color(scale=alt.Scale(scheme='Red')),
                                alt.value('lightgray'))

# create the horizontal bar chart with selection
bar = alt.Chart(state_counts).mark_bar(color = 'Black').encode(
    y=alt.Y('State:N', title='State'),
    x=alt.X('Crash Count:Q', title='Crash Count'),
    color=color_condition,
    tooltip=['State_name', 'Crash Count']
).properties(
    width=500,
    height=400,
    title={
        "text": ["Total Crash Count by State"],
        "subtitle": ["(Select a state to view fatalities over years)"],
        "fontSize": 16,
        "fontWeight": "bold",
        "subtitleFontSize": 12,
        "subtitleFontWeight": "normal"
    }
).add_selection(state_selection)

# convert the date column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# group the data by state and year, and calculate the total fatalities
state_year_counts = data.groupby(['State', pd.Grouper(key='Date', freq='Y')])['Fatalities'].sum().reset_index()

# create the line plot
line = alt.Chart(state_year_counts).mark_line(color = 'Red').encode(
    x=alt.X('Date:T', title='Year'),
    y=alt.Y('Fatalities:Q', title='Fatalities'),
    color=alt.Color('State:N', title='State'),
    tooltip=['State', 'Date', 'Fatalities']
).properties(
    width=600,
    height=400,
    title={
        "text": ["Number of Fatalities for each State over Years"],
        "fontSize": 16,
        "fontWeight": "bold"
    }
).transform_filter(
    state_selection
)

# display the combined chart
(bar | line).resolve_scale(color='shared')


/Users/sangeetha/opt/anaconda3/envs/anly503/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)